## Imports

In [1]:
import os
import numpy as np

import torch
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 28*28

# Other
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GRAYSCALE = True

### CIFAR10 Dataset

In [3]:
transform = (
    transforms.Compose([
        #transforms.Resize(224),
        #transforms.RandomHorizontalFlip(p=.40),
        #transforms.RandomRotation(30),
        transforms.ToTensor(),
        #transforms.Normalize(
        #    (0.4914, 0.4822, 0.4465), 
        #    (0.2471, 0.2435, 0.2616)
        #)
    ])
)

In [4]:
##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = MNIST(root='data', 
                     train=True, 
                     transform=transform, 
                     download=True)

test_dataset = MNIST(root='data', 
                     train=False, 
                     transform=transform)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print(DEVICE)
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

cuda
Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [5]:
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):
    for batch_idx, (x, y) in enumerate(train_loader):
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


## Model

In [6]:
from google.colab import drive


drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import sys


PATH = "/content/drive/My Drive/Colab Notebooks/EECS553ML_reproduce/"
#PATH = "/content/drive/My Drive/EECS553ML_reproduce/"
sys.path.append(PATH)

In [8]:
!cd "/content/drive/My Drive/EECS553ML_reproduce/";ls

/bin/bash: line 0: cd: /content/drive/My Drive/EECS553ML_reproduce/: No such file or directory
data  drive  sample_data


In [9]:
from resnet18 import ResNet152


torch.manual_seed(RANDOM_SEED)
model = ResNet152(size_for_cifar=False)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

## Training

In [10]:
from trainer import Trainer


trainer = Trainer(model, train_loader, DEVICE)

Training on cuda


In [11]:
trainer.train(LEARNING_RATE, NUM_EPOCHS)

Epoch: 001/010 | Batch 0000/0469 | Cost: 5.3727
Epoch: 001/010 | Batch 0050/0469 | Cost: 0.4821
Epoch: 001/010 | Batch 0100/0469 | Cost: 0.1062
Epoch: 001/010 | Batch 0150/0469 | Cost: 0.0829
Epoch: 001/010 | Batch 0200/0469 | Cost: 0.2100
Epoch: 001/010 | Batch 0250/0469 | Cost: 0.0989
Epoch: 001/010 | Batch 0300/0469 | Cost: 0.0630
Epoch: 001/010 | Batch 0350/0469 | Cost: 0.0542
Epoch: 001/010 | Batch 0400/0469 | Cost: 0.0214
Epoch: 001/010 | Batch 0450/0469 | Cost: 0.0256
Epoch: 001/010 | Train: 96.050%
Time elapsed: 8.76 min
Epoch: 002/010 | Batch 0000/0469 | Cost: 0.0417
Epoch: 002/010 | Batch 0050/0469 | Cost: 0.0408
Epoch: 002/010 | Batch 0100/0469 | Cost: 0.0163
Epoch: 002/010 | Batch 0150/0469 | Cost: 0.0807
Epoch: 002/010 | Batch 0200/0469 | Cost: 0.1597
Epoch: 002/010 | Batch 0250/0469 | Cost: 0.1255
Epoch: 002/010 | Batch 0300/0469 | Cost: 0.1430
Epoch: 002/010 | Batch 0350/0469 | Cost: 0.0377
Epoch: 002/010 | Batch 0400/0469 | Cost: 0.0919
Epoch: 002/010 | Batch 0450/0469 

## Evaluation

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (trainer.compute_accuracy(test_loader)))

In [ ]:
#%watermark -iv

In [ ]:
torch.save(model.state_dict(), f"{PATH}mnist_resnet152_model.t7")